In [2]:
pip install hyperopt

  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:0000:0100:01
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import re
import datetime
import dateutil.relativedelta
from datetime import datetime
import math
from sklearn.model_selection import train_test_split

import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators import H2OGradientBoostingEstimator

from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

import functions

In [4]:
from functions import rnkOrdering2,rnkOrdering2_con_wt,rnkOrdering2_ks_code,rnkOrderingN

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2019-01-15; OpenJDK Runtime Environment 18.9 (build 11.0.2+9); OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpo719t584
  JVM stdout: /tmp/tmpo719t584/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpo719t584/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_unknownUser_oyryg1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.471 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [9]:
data = pd.read_csv('s3://scien-ds-sm-root-backup/home/vikashagarwal/LNV6/LNV6_Binned_Data_23Sept2024.csv')
data.shape

(3617, 367)

In [10]:
data.head(1)

,unique_id,p_inpnamefirstflag,p_inpnamelastflag,p_inpaddrstflag,p_inpaddrcityflag,p_inpaddrstateflag,p_inpaddrzipflag,p_inpssnflag,p_inpssnlength,p_inpdobflag,p_inpphonehomeflag,p_inpdlflag,p_inpdlstateflag,p_inplexidflag,p_inpvalnameinvalidflag,p_inpvaladdrstinvalidflag,p_inpvalssninvalidflag,p_inpvalssnprobindx,p_inpvaldobinvalidflag,p_inpvalphonehomeinvalidflag,p_inpvaldlinvalidflag,p_inpvaldlstateinvalidflag,p_rvunscorableflag,p_lexidisdeceasedflag,pl_bestdobage,pl_srcbureauflag1y,pl_srcbureauflagev,pl_srcbureauoldmsncev,pl_srcbureauonlyflagev,pl_srccnt1y,pl_srccntev,pl_srccredcnt1y,pl_srccredcntev,pl_srccredoldmsncev,pl_srcnondrgcnt1y,pl_srcnondrgcntev,pl_srcnondrgoldmsncev,pl_srcoldmsncev,pl_srcvoterflagev,pl_srcvoteroldmsncev,pl_ssncnt5y,pl_ssnnewcnt6m,pl_inpnamebureaumatchindx,pl_inpnamecredmatchindx,pl_inpnamematchindx,pl_inpnamenondrgmatchindx,pi_inpaddravmval,pi_inpaddravmratio1y,pi_inpaddravmratio5y,pi_inpaddrisaptflag,pi_inpaddrisbusinessflag,pi_inpaddriscollegeflag,pi_inpaddrispoboxflag,pi_inpaddrissfdflag,pi_inpaddrisvacantflag,pi_inpaddrmailmatchflag,pi_inpaddrmedavmctyratio,pi_inpaddrtaxval,pi_inpaddrtaxvalyr,pi_inpaddrmktval,pi_inpaddrnamepropmatchindx,pi_inpaddrpurchamt,pi_inpaddrpurchmsncev,pi_inpaddrtype,pi_inpaddruniqmaildlvryflag,pl_inpaddrmiskeyflag,pl_inpaddrsrcbureauflagev,pl_inpaddrsrcbureauoldmsncev,pl_inpaddrsrcbureauonlyflagev,pl_inpaddrsrccntev,pl_inpaddrsrccredcntev,pl_inpaddrsrccredoldmsncev,pl_inpaddrsrcnondrgcntev,pl_inpaddrsrcnondrgoldmsncev,pl_inpaddrsrcoldmsncev,pl_lexidpersubjaddrcnt6m,pl_lexidpersubjaddrsfdcnt6m,pl_phonepersubjaddrcnt6m,pl_phonepersubjaddrsfdcnt6m,pl_fileaddrflag,pl_fileaddravmval,pl_fileaddravmratio1y,pl_fileaddravmratio5y,pl_fileaddrisaptflag,pl_fileaddrisbusinessflag,pl_fileaddriscollegeflag,pl_fileaddrissfdflag,pl_fileaddrisvacantflag,pl_fileaddrmailmatchflag,pl_fileaddrmedavmctyratio,pl_fileaddrtaxval,pl_fileaddrtaxvalyr,pl_fileaddrmktval,pl_fileaddrnamepropmatchindx,pl_fileaddrpurchamt,pl_fileaddrpurchmsncev,pl_fileaddrtype,pl_fileaddruniqmaildlvryflag,pl_addrcnt5y,pl_addrcollegeflag,pl_addremrgcnt1y,pl_addremrgcnt6m,pl_addremrgcnt7y,pl_addrmaxmoncnt,pl_addrlastmoveaddrtype1y,pl_addrlastmoveaddrtype5y,pl_addrlastmoveavmratio1y,pl_addrlastmoveavmratio5y,pl_addrlastmovedist1y,pl_addrlastmovedist5y,pl_addrlastmoveflag1y,pl_addrlastmoveflag5y,pl_addrlastmovemsnc5y,pl_addrmovecurrtoinpflag,pl_addrmovecurrtoinpaddrtype,pl_addrmovecurrtoinpavmratio,pl_addrmovecurrtoinpdist,pl_addrmoveprevtocurrflag1y,pl_addrmoveprevtocurrflag5y,pl_addrmoveprevtocurraddrtype1y,pl_addrmoveprevtocurraddrtype5y,pl_addrmoveprevtocurravmratio1y,pl_addrmoveprevtocurravmratio5y,pl_addrmoveprevtocurrdist1y,pl_addrmoveprevtocurrdist5y,pl_addrmoveprevtocurrmsnc5y,pl_addrprevandcurrsubjresspanev,pi_inpssnisdeceasedflag,pi_lexidperinpssncnt6m,pl_bestssnappendflag,pl_inpssnindx,pl_inpssnmiskeyflag,pl_inpssnsrcbureauflagev,pl_inpssnsrcbureauoldmsncev,pl_inpssnsrcbureauonlyflagev,pl_inpssnsrccntev,pl_inpssnsrccredcntev,pl_inpssnsrccredoldmsncev,pl_inpssnsrcnondrgcntev,pl_inpssnsrcnondrgoldmsncev,pl_inpssnsrcoldmsncev,pi_inpdobage,pl_inpdobmiskeyflag,pl_inpdobsrcbureauflagev,pl_inpdobsrcbureauoldmsncev,pl_inpdobsrcbureauonlyflagev,pl_inpdobsrccntev,pl_inpdobsrccredcntev,pl_inpdobsrccredoldmsncev,pl_inpdobsrcnondrgcntev,pl_inpdobsrcnondrgoldmsncev,pl_inpdobsrcoldmsncev,pi_inpphoneisbusphoneflag,pl_inpphonemiskeyflag,pl_inpphonesrccntev,pl_inpphonesrcoldmsncev,pl_lexidpersubjphonecnt6m,pl_educollrecflagev,pl_educolllevelhighev,pl_educollevidflagev,pl_educolltierbestev,pl_proflicactvnewindx,pl_profliccntev,pl_proflicactvcnt,pl_astcntev,pl_astcnt2y,pl_astemrgnewmsncev,pl_astoldmsncev,pl_astpropownershipindx,pl_astpropcntev,pl_astpropcurrcnt,pl_astproppurchcntev,pl_astproppurchcnt5y,pl_astproppurchcnt1y,pl_astpropsalecnt1y,pl_astpropsalecnt5y,pi_inpaddrsubjownedflag,pi_inpaddrsubjownedavmval,pi_inpaddrsubjownedtaxval,pi_inpaddrsubjownedmktval,pi_inpaddrsubjownedpurchamt,pi_inpaddrsubjownedpurchmsncev,pl_filead

In [11]:
overall_target_mean = np.mean(data['bad_flag'])
overall_target_mean

0.021288360519767763

In [12]:
train, test= train_test_split(data, test_size= 0.3, random_state= 2, shuffle= True, stratify= data['bad_flag'])

In [13]:
train_target_mean = train.bad_flag.mean()
test_target_mean = test.bad_flag.mean()
train_target_mean,test_target_mean

(0.02133544053733702, 0.02117863720073665)

In [14]:
Train_h2o = h2o.H2OFrame(train)
Test_h2o = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [15]:
Train_h2o.shape,Test_h2o.shape

((2531, 367), (1086, 367))

In [16]:
if 'wt2' not in Train_h2o.columns:
    Train_h2o['wt2'] = 1
    Test_h2o['wt2'] = 1

In [17]:
Train_h2o[0,:]

unique_id,p_inpnamefirstflag,p_inpnamelastflag,p_inpaddrstflag,p_inpaddrcityflag,p_inpaddrstateflag,p_inpaddrzipflag,p_inpssnflag,p_inpssnlength,p_inpdobflag,p_inpphonehomeflag,p_inpdlflag,p_inpdlstateflag,p_inplexidflag,p_inpvalnameinvalidflag,p_inpvaladdrstinvalidflag,p_inpvalssninvalidflag,p_inpvalssnprobindx,p_inpvaldobinvalidflag,p_inpvalphonehomeinvalidflag,p_inpvaldlinvalidflag,p_inpvaldlstateinvalidflag,p_rvunscorableflag,p_lexidisdeceasedflag,pl_bestdobage,pl_srcbureauflag1y,pl_srcbureauflagev,pl_srcbureauoldmsncev,pl_srcbureauonlyflagev,pl_srccnt1y,pl_srccntev,pl_srccredcnt1y,pl_srccredcntev,pl_srccredoldmsncev,pl_srcnondrgcnt1y,pl_srcnondrgcntev,pl_srcnondrgoldmsncev,pl_srcoldmsncev,pl_srcvoterflagev,pl_srcvoteroldmsncev,pl_ssncnt5y,pl_ssnnewcnt6m,pl_inpnamebureaumatchindx,pl_inpnamecredmatchindx,pl_inpnamematchindx,pl_inpnamenondrgmatchindx,pi_inpaddravmval,pi_inpaddravmratio1y,pi_inpaddravmratio5y,pi_inpaddrisaptflag,pi_inpaddrisbusinessflag,pi_inpaddriscollegeflag,pi_inpaddrispoboxflag,pi_inpaddrissfdflag,pi_inpaddrisvacantflag,pi_inpaddrmailmatchflag,pi_inpaddrmedavmctyratio,pi_inpaddrtaxval,pi_inpaddrtaxvalyr,pi_inpaddrmktval,pi_inpaddrnamepropmatchindx,pi_inpaddrpurchamt,pi_inpaddrpurchmsncev,pi_inpaddrtype,pi_inpaddruniqmaildlvryflag,pl_inpaddrmiskeyflag,pl_inpaddrsrcbureauflagev,pl_inpaddrsrcbureauoldmsncev,pl_inpaddrsrcbureauonlyflagev,pl_inpaddrsrccntev,pl_inpaddrsrccredcntev,pl_inpaddrsrccredoldmsncev,pl_inpaddrsrcnondrgcntev,pl_inpaddrsrcnondrgoldmsncev,pl_inpaddrsrcoldmsncev,pl_lexidpersubjaddrcnt6m,pl_lexidpersubjaddrsfdcnt6m,pl_phonepersubjaddrcnt6m,pl_phonepersubjaddrsfdcnt6m,pl_fileaddrflag,pl_fileaddravmval,pl_fileaddravmratio1y,pl_fileaddravmratio5y,pl_fileaddrisaptflag,pl_fileaddrisbusinessflag,pl_fileaddriscollegeflag,pl_fileaddrissfdflag,pl_fileaddrisvacantflag,pl_fileaddrmailmatchflag,pl_fileaddrmedavmctyratio,pl_fileaddrtaxval,pl_fileaddrtaxvalyr,pl_fileaddrmktval,pl_fileaddrnamepropmatchindx,pl_fileaddrpurchamt,pl_fileaddrpurchmsncev,pl_fileaddrtype,pl_fileaddruniqmaildlvryflag,pl_addrcnt5y,pl_addrcollegeflag,pl_addremrgcnt1y,pl_addremrgcnt6m,pl_addremrgcnt7y,pl_addrmaxmoncnt,pl_addrlastmoveaddrtype1y,pl_addrlastmoveaddrtype5y,pl_addrlastmoveavmratio1y,pl_addrlastmoveavmratio5y,pl_addrlastmovedist1y,pl_addrlastmovedist5y,pl_addrlastmoveflag1y,pl_addrlastmoveflag5y,pl_addrlastmovemsnc5y,pl_addrmovecurrtoinpflag,pl_addrmovecurrtoinpaddrtype,pl_addrmovecurrtoinpavmratio,pl_addrmovecurrtoinpdist,pl_addrmoveprevtocurrflag1y,pl_addrmoveprevtocurrflag5y,pl_addrmoveprevtocurraddrtype1y,pl_addrmoveprevtocurraddrtype5y,pl_addrmoveprevtocurravmratio1y,pl_addrmoveprevtocurravmratio5y,pl_addrmoveprevtocurrdist1y,pl_addrmoveprevtocurrdist5y,pl_addrmoveprevtocurrmsnc5y,pl_addrprevandcurrsubjresspanev,pi_inpssnisdeceasedflag,pi_lexidperinpssncnt6m,pl_bestssnappendflag,pl_inpssnindx,pl_inpssnmiskeyflag,pl_inpssnsrcbureauflagev,pl_inpssnsrcbureauoldmsncev,pl_inpssnsrcbureauonlyflagev,pl_inpssnsrccntev,pl_inpssnsrccredcntev,pl_inpssnsrccredoldmsncev,pl_inpssnsrcnondrgcntev,pl_inpssnsrcnondrgoldmsncev,pl_inpssnsrcoldmsncev,pi_inpdobage,pl_inpdobmiskeyflag,pl_inpdobsrcbureauflagev,pl_inpdobsrcbureauoldmsncev,pl_inpdobsrcbureauonlyflagev,pl_inpdobsrccntev,pl_inpdobsrccredcntev,pl_inpdobsrccredoldmsncev,pl_inpdobsrcnondrgcntev,pl_inpdobsrcnondrgoldmsncev,pl_inpdobsrcoldmsncev,pi_inpphoneisbusphoneflag,pl_inpphonemiskeyflag,pl_inpphonesrccntev,pl_inpphonesrcoldmsncev,pl_lexidpersubjphonecnt6m,pl_educollrecflagev,pl_educolllevelhighev,pl_educollevidflagev,pl_educolltierbestev,pl_proflicactvnewindx,pl_profliccntev,pl_proflicactvcnt,pl_astcntev,pl_astcnt2y,pl_astemrgnewmsncev,pl_astoldmsncev,pl_astpropownershipindx,pl_astpropcntev,pl_astpropcurrcnt,pl_astproppurchcntev,pl_astproppurchcnt5y,pl_astproppurchcnt1y,pl_astpropsalecnt1y,pl_astpropsalecnt5y,pi_inpaddrsubjownedflag,pi_inpaddrsubjownedavmval,pi_inpaddrsubjownedtaxval,pi_inpaddrsubjownedmktval,pi_inpaddrsubjownedpurchamt,pi_inpaddrsubjownedpurchmsncev,pl_fileadd

In [18]:
bad_col = "bad_flag"
unique_cols = ['unique_id', 'funded_flag']
wt_col = "wt2"

In [19]:
Train_h2o[bad_col] = Train_h2o[bad_col].asfactor() # Convert column in the current frame to categoricals.
folds = 10
predictors = Train_h2o.columns
predictors.remove('unique_id')
predictors.remove('funded_flag')
len(predictors)

366

In [20]:
def Bucket_Order(data):
    data['lower_bound'] = data['Bucket'].str.replace('[','').str.replace(']','').str.replace('(','').str.replace(')','').str.split(',').str[0]
    data['lower_bound'] = data['lower_bound'].apply(lambda x: -np.inf if x == '-inf' else np.inf if x == 'inf' else float(x))
    data = data.sort_values('lower_bound')
    a = data.loc[data['lower_bound']=="[-99999, -99998, -99997]"]
    b = data.loc[data['lower_bound']!="[-99999, -99998, -99997]"]
    b['Is_Trend']= 'Yes' if (
            (b['Train_BadRate'].is_monotonic_decreasing and b['Test_BadRate'].is_monotonic_decreasing) or
            (b['Train_BadRate'].is_monotonic_increasing and b['Test_BadRate'].is_monotonic_increasing)
        ) else 'No'
    data = pd.concat(objs=[a,b])
    data = data.drop(columns=['lower_bound'])
    return data

In [21]:
df_bv= pd.DataFrame()

for col in predictors:
    if col in ['unique_id','wt2','bad_flag'] + ['p_inpvaldlinvalidflag','p_inpvaldlstateinvalidflag','pl_drglienotheramttot7y','pl_drglienothernewmsnc7y']:  ## These 4 columns have all their values as either -99999, -99998 or -99997
        continue

    ### Train
    temp_tr= train.groupby(col).agg(bad_rate_tr= ('bad_flag', 'mean'),
                                      bad_tr= ('bad_flag', 'sum'),
                                     volume_tr= ('unique_id', 'count')).reset_index()
    temp_tr.columns= ['Bucket', 'Train_BadRate','Train_Bad','Train_#_Apps']
    temp_tr['Variable']= col
    temp_tr['Train_%_Apps'] = temp_tr['Train_#_Apps'] / temp_tr['Train_#_Apps'].sum()

    ### Test
    temp_te= test.groupby(col).agg(bad_rate_te= ('bad_flag', 'mean'),
                                     bad_te= ('bad_flag', 'sum'),
                                    volume_tr= ('unique_id', 'count')).reset_index()
    temp_te.columns= ['Bucket', 'Test_BadRate','Test_Bad','Test_#_Apps']
    temp_te['Test_%_Apps'] = temp_te['Test_#_Apps'] / temp_te['Test_#_Apps'].sum()

    ### Concatenating
    temp= temp_tr.merge(temp_te, on= ['Bucket'], how= 'left')
    temp = Bucket_Order(temp)
    
#     display(temp)
    df_bv= pd.concat([df_bv, temp]).reset_index(drop= True)
    df_bv = df_bv[['Variable','Bucket','Train_BadRate','Train_Bad','Train_#_Apps','Train_%_Apps','Test_BadRate','Test_Bad','Test_#_Apps','Test_%_Apps','Is_Trend']]
    del temp, temp_tr, temp_te

In [22]:
print(df_bv.shape)
display(df_bv.head())

(1197, 11)


,Variable,Bucket,Train_BadRate,Train_Bad,Train_#_Apps,Train_%_Apps,Test_BadRate,Test_Bad,Test_#_Apps,Test_%_Apps,Is_Trend
0,p_inpnamefirstflag,"[1.0, inf)",0.021335,54.0,2531,1.0,0.021179,23.0,1086.0,1.0,Yes
1,p_inpnamelastflag,"[1.0, inf)",0.021335,54.0,2531,1.0,0.021179,23.0,1086.0,1.0,Yes
2,p_inpaddrstflag,"[1.0, inf)",0.021335,54.0,2531,1.0,0.021179,23.0,1086.0,1.0,Yes
3,p_inpaddrcityflag,"[1.0, inf)",0.021335,54.0,2531,1.0,0.021179,23.0,1086.0,1.0,Yes
4,p_inpaddrstateflag,"[1.0, inf)",0.021335,54.0,2531,1.0,0.021179,23.0,1086.0,1.0,Yes


In [23]:
# df_bv.to_excel('LN_V6_Variables_Trend.xlsx', index=False)

In [24]:
Selected_Columns = list(set(df_bv[df_bv['Is_Trend']=='Yes']['Variable'].to_list()))
print(len(Selected_Columns))
# Selected_Columns

61


In [25]:
Rejected_Columns = list(set(df_bv[df_bv['Is_Trend']=='No']['Variable'].to_list()))
print(len(Rejected_Columns))
# Rejected_Columns

299


In [26]:
model = H2OXGBoostEstimator(ntrees = 50, seed = 42)

In [27]:
model.train(y = bad_col, x = Selected_Columns, training_frame = Train_h2o, weights_column = wt_col)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1728625371034_1


Model Summary: 
    number_of_trees
--  -----------------
    50

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.014398660841865694
RMSE: 0.11999442004470746
LogLoss: 0.05947984261310278
Mean Per-Class Error: 0.22666307809626338
AUC: 0.9365570657456003
AUCPR: 0.5653535887814335
Gini: 0.8731141314912005

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.15989677608013153
       0     1    Error    Rate
-----  ----  ---  -------  -------------
0      2455  22   0.0089   (22.0/2477.0)
1      24    30   0.4444   (24.0/54.0)
Total  2479  52   0.0182   (46.0/2531.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.159897     0.566038  35
max f2                       0.129132     0.580205  46
max f0point5                 0.264508     0.64433   25
max accuracy                 0.264508     0.984591  25
max precision                0.848203     1         0
max recall                   0.00688653   1         299
max specificity              0.848203     1         0
max absolute_mcc             0.264508     0.567866  25
max min_per_class_accuracy   0.0258489    0.84134   171
max mean_per_class_accuracy  0.0392767    0.858558  130
max tns                      0.848203     2477      0
max fns                      0.848203     53        0
max fps                      0.000213489  2477      399
max tps                      0.00688653   54        299
max tnr                      0.848203     1         0
max fnr                      0.848203     0.981481  0
max fpr                      0.000213489  1         399
max tpr                      0.00688653   1         299

Gains/Lift Table: Avg response rate:  2.13 %, avg score:  2.15 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0102726                   0.310133           34.2514   34.2514            0.730769         0.44736      0.730769                    0.44736             0.351852        0.351852                   3325.14   3325.14            0.349026
2        0.0205452                   0.159897           19.8298   27.0406            0.423077         0.229898     0.576923                    0.338629            0.203704        0.555556                   1882.98   2604.06            0.546674
3        0.0304228                   0.129132           7.49926   20.696             0.16             0.139287     0.441558                    0.273908            0.0740741       0.62963                    649.926   1969.6             0.61227
4        0.0403003                   0.088984           1.87481   16.083             0.04             0.105214     0.343137                    0.232561            0.0185185       0.648148                   87.4815   1508.3             0.621099
5        0.0557092                   0.0674507          4.80722   12.9641            0.102564         0.0727961    0.276596                    0.188371            0.0740741       0.722222                   380.722   1196.41            0.681043
6        0.100356                    0.0444931          1.65913   7.93475            0.0353982        0.0567968    0.169291                    0.129836            0.0740741       0.796296                   65.9128   693.475            0.711113
7        0.173844                    0.0258487          1.00796   5.00661            0.021

### TRAIN

In [28]:
predict = model.predict(Train_h2o)
p =predict.as_data_frame()
dd = Train_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
# rnkOrdering2_ks_train_nw = rnkOrdering2_ks_code(pred, bad_col, "conwt","p1", 5)
print(ks_dict)
ks_df

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
{'Gini': 74.9, 'KS': 68.47}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.020535,0.848203,0.079250,507,460,47,9.27,20.03,18.57,87.04,68.47,1.345036
2,0.011035,0.020363,0.014803,506,501,5,0.99,40.02,38.80,96.30,57.50,0.490311
3,0.006872,0.011035,0.008509,506,504,2,0.40,60.02,59.14,100.00,40.86,0.450948
4,0.002054,0.006800,0.003528,506,506,0,0.00,80.01,79.57,100.00,20.43,0.352798
5,0.000205,0.002054,0.001118,506,506,0,0.00,100.00,100.00,100.00,0.00,0.111796


### TEST

In [29]:
predict = model.predict(Test_h2o)
p =predict.as_data_frame()
dd = Test_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
print(ks_dict)
ks_df

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
{'Gini': 28.349999999999998, 'KS': 23.91}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.021552,0.483783,0.069226,218,208,10,4.59,20.07,19.57,43.48,23.91,2.332624
2,0.011035,0.021408,0.014480,217,214,3,1.38,40.06,39.70,56.52,16.82,0.067989
3,0.006667,0.011035,0.008455,217,212,5,2.30,60.04,59.64,78.26,18.62,1.454471
4,0.002067,0.006667,0.003578,217,214,3,1.38,80.02,79.77,91.30,11.53,1.022154
5,0.000228,0.002057,0.001145,217,215,2,0.92,100.00,100.00,100.00,0.00,0.805459


In [30]:
var_imp = model._model_json['output']['variable_importances'].as_data_frame()
var_imp['Var_unique'] = var_imp['variable'].str.split('.').str[0]
# var_imp

In [31]:
var_imp_list_baseline1 = list(var_imp['Var_unique'].unique())
print(len(var_imp_list_baseline1))
# print(var_imp_list_baseline1)

31


In [32]:
Var_Imp_DF = var_imp.groupby('Var_unique').agg({'percentage':'sum'}).reset_index().sort_values(by='percentage',ascending=False).reset_index(drop=True)
Var_Imp_DF.head()

,Var_unique,percentage
0,pl_inpdobsrcnondrgcntev,0.164508
1,pl_inpssnmiskeyflag,0.086497
2,pi_inpaddrisaptflag,0.083426
3,pl_addrmovecurrtoinpflag,0.071595
4,pi_inpaddrissfdflag,0.069961


## HyperOpt

In [33]:
from IPython.display import clear_output

def get_predictions(data_h2o, bad_col, wt_col, classifier):
    predict = classifier.predict(data_h2o)
    p = predict.as_data_frame()
    dd = data_h2o[:, unique_cols + [bad_col] + [wt_col]].as_data_frame()
    dd['RECWGT'] = dd[wt_col]
    dd['con_wt'] = 1
    pred = pd.concat([dd, p], axis=1)
    return pred

def objective(space):
    ks_data = pd.DataFrame()
    from sklearn.metrics import roc_auc_score

    classifier = H2OXGBoostEstimator(ntrees = int(space['n_estimators']),
                                max_depth = int(space['max_depth']),
                                learn_rate = space['learning_rate'],
                                min_rows = space['min_child_weight'],
                                # sample_rate = space['subsample'],
                                # col_sample_rate_per_tree = space['colsample_bytree'],
                                reg_alpha = space['reg_alpha'],
                                reg_lambda = space['reg_lambda'],
                                min_split_improvement = space['gamma'],
                                # col_sample_rate = space['colsample_bylevel'],
                                seed=space['seed'],
                                model_id="Scienaptic_LN_V6_Model_1",
#                                 nfolds = 10,
#                                 distribution = "bernoulli",
#                                 stopping_tolerance=0.000001, 
#                                 stopping_rounds=20,
#                                 stopping_metric = 'logloss',
#                                 categorical_encoding ='sort_by_response'
                                )
    
    h2o.no_progress()
    classifier.train(y=bad_col, x = predictors, training_frame=Train_h2o, weights_column =wt_col)
    # h2o.show_progress()
    
    ################        TRAIN      ##########################
    
    pred = get_predictions(Train_h2o, bad_col, wt_col, classifier)

    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_train = roc_auc_score(y_true, y_prob, sample_weight=weights)
    
    NonWeighted_Dict, NonWeighted_KS_Table = rnkOrderingN(pred, bad_col, "con_wt","p1", 5) # Non Weighted
    # Weighted_Dict, Weighted_KS_Table = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5) # Weighted
    
    Is_NW_Train_Bad_Rate_RankOrder = NonWeighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
    NonWeighted_KS_Table['on'] = 'Train'
    NonWeighted_KS_Table['iter'] = len(trials.trials)
    NonWeighted_KS_Table['bins'] = len(NonWeighted_KS_Table)
#     NonWeighted_KS_Table['Weighted'] = 'No'
    
#     Is_Train_Bad_Rate_RankOrder = Weighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
#     Weighted_KS_Table['on'] = 'Train'
#     Weighted_KS_Table['iter'] = len(trials.trials)
#     Weighted_KS_Table['bins'] = len(Weighted_KS_Table)
#     Weighted_KS_Table['Weighted'] = 'Yes'
    # ks_data = pd.concat([ks_data,NonWeighted_KS_Table,Weighted_KS_Table])
    
    ks_data = pd.concat([ks_data,NonWeighted_KS_Table])
    
#     train_ks = Weighted_Dict['KS']
#     train_gini = Weighted_Dict['Gini']

    NW_train_ks = NonWeighted_Dict['KS']
    NW_train_gini = NonWeighted_Dict['Gini']
    
     ################        TEST       ##########################
    
    pred = get_predictions(Test_h2o, bad_col, wt_col, classifier)
    
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_test = roc_auc_score(y_true, y_prob, sample_weight=weights)
    
    NonWeighted_Dict, NonWeighted_KS_Table = rnkOrderingN(pred, bad_col, "con_wt","p1", 5) # Non Weighted
    # Weighted_Dict, Weighted_KS_Table = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5) # Weighted
    
    Is_NW_Test_Bad_Rate_RankOrder = NonWeighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
    NonWeighted_KS_Table['on'] = 'Test'
    NonWeighted_KS_Table['iter'] = len(trials.trials)
    NonWeighted_KS_Table['bins'] = len(NonWeighted_KS_Table)
#     NonWeighted_KS_Table['Weighted'] = 'No'
    
#     Is_Test_Bad_Rate_RankOrder = Weighted_KS_Table['Bad_Rate'].is_monotonic_decreasing
#     Weighted_KS_Table['on'] = 'Test'
#     Weighted_KS_Table['iter'] = len(trials.trials)
#     Weighted_KS_Table['bins'] = len(Weighted_KS_Table)
#     Weighted_KS_Table['Weighted'] = 'Yes'
    # ks_data = pd.concat([ks_data,NonWeighted_KS_Table,Weighted_KS_Table])
    
    ks_data = pd.concat([ks_data,NonWeighted_KS_Table])
    
    # test_ks = Weighted_Dict['KS']
    # test_gini = Weighted_Dict['Gini']

    NW_test_ks = NonWeighted_Dict['KS']
    NW_test_gini = NonWeighted_Dict['Gini']

    # KS_diff = abs(train_ks-test_ks)
    # gini_diff = abs(train_gini-test_gini)
    NW_ks_diff = abs(NW_train_ks-NW_test_ks)
    NW_gini_diff = abs(NW_train_gini-NW_test_gini)
    
    print(f'{len(trials.trials)}, ks_diff_nw: {NW_ks_diff}')

    auc_diff = abs(auc_train-auc_test) 
    
    loss = NW_ks_diff
    
    return{'loss':NW_ks_diff, 'status': STATUS_OK ,
               # 'ks_diff':KS_diff,'train_ks':train_ks,'test_ks':test_ks, #'oot_ks':oot_ks,    ###  WEIGHTED KS
               # 'gini_diff':gini_diff,'train_gini':train_gini,'test_gini':test_gini, #'oot_gini':oot_gini,     ###  WEIGHTED GINI
               'NW_ks_diff':NW_ks_diff,'NW_train_ks':NW_train_ks,'NW_test_ks':NW_test_ks,# 'oot_ks_cw':oot_ks_cw,   ### NON_WEIGHTED KS
               'NW_gini_diff':NW_gini_diff,'NW_train_gini':NW_train_gini,'NW_test_gini':NW_test_gini #'oot_gini_cw':oot_gini_cw   ###   NON-WEIGHTED GINI
              }

In [34]:
space = {
    'max_depth':2,
    'learning_rate' : hp.quniform('learning_rate',0.01,0.05,0.001),
    'n_estimators' : ho_scope.int(hp.quniform('n_estimators',100,1000,25)),
    'min_child_weight' : hp.quniform('min_child_weight', 2, 32, 2),
    # 'subsample' : hp.quniform('subsample', 0.1, 0.6, 0.05),
    # 'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'reg_lambda':  hp.quniform('reg_lambda',1,15,0.5),
    'reg_alpha': hp.quniform('reg_alpha',1,10,0.5),
    'gamma':hp.quniform('gamma',1,25,0.5),
    # 'colsample_bylevel':hp.quniform('colsample_bylevel',0.5, 1, 0.05),
    'seed':42
}

In [35]:
trials = Trials()

In [49]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)

print("Best: ", best)

1, ks_diff_nw: 3.2300000000000004                     
2, ks_diff_nw: 3.2300000000000004                                               
3, ks_diff_nw: 3.2300000000000004                                               
4, ks_diff_nw: 3.2300000000000004                                               
5, ks_diff_nw: 7.450000000000003                                                
6, ks_diff_nw: 3.2300000000000004                                               
7, ks_diff_nw: 6.789999999999999                                                
8, ks_diff_nw: 3.2300000000000004                                               
9, ks_diff_nw: 5.589999999999996                                                
10, ks_diff_nw: 3.2300000000000004                                              
11, ks_diff_nw: 3.2300000000000004                                               
12, ks_diff_nw: 3.2300000000000004                                               
13, ks_diff_nw: 3.2300000000000004                  

In [50]:
best

{'gamma': 12.0,
 'learning_rate': 0.028,
 'min_child_weight': 26.0,
 'n_estimators': 825.0,
 'reg_alpha': 1.0,
 'reg_lambda': 3.5}

In [51]:
tr_results = pd.DataFrame(trials.results).reset_index()
tr_results['iter'] = tr_results['index']+1
print(f'TR_RESULTS = {tr_results.shape}')
display(tr_results.head())

tr_vals = pd.DataFrame(trials.vals).reset_index()
tr_vals['iter'] = tr_vals['index']+1
print(f'TR_VALUES = {tr_vals.shape}')
display(tr_vals.head())

trials_full = pd.merge(tr_vals.drop('index',axis=1),tr_results.drop('index',axis=1),how='left',on='iter')
print(f'Trials Full = {trials_full.shape}')
display(trials_full.head())

TR_RESULTS = (30, 10)


,index,loss,status,NW_ks_diff,NW_train_ks,NW_test_ks,NW_gini_diff,NW_train_gini,NW_test_gini,iter
0,0,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07,1
1,1,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07,2
2,2,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07,3
3,3,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07,4
4,4,7.45,ok,7.45,40.47,47.92,2.80,52.19,54.99,5


TR_VALUES = (30, 8)


,index,gamma,learning_rate,min_child_weight,n_estimators,reg_alpha,reg_lambda,iter
0,0,24.0,0.014,16.0,925.0,2.0,8.0,1
1,1,22.0,0.018,10.0,175.0,2.5,8.0,2
2,2,20.5,0.012,14.0,300.0,8.5,4.0,3
3,3,19.5,0.038,26.0,350.0,2.0,11.0,4
4,4,3.0,0.011,6.0,950.0,9.0,14.5,5


Trials Full = (30, 15)


,gamma,learning_rate,min_child_weight,n_estimators,reg_alpha,reg_lambda,iter,loss,status,NW_ks_diff,NW_train_ks,NW_test_ks,NW_gini_diff,NW_train_gini,NW_test_gini
0,24.0,0.014,16.0,925.0,2.0,8.0,1,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07
1,22.0,0.018,10.0,175.0,2.5,8.0,2,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07
2,20.5,0.012,14.0,300.0,8.5,4.0,3,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07
3,19.5,0.038,26.0,350.0,2.0,11.0,4,3.23,ok,3.23,3.01,6.24,1.55,1.48,-0.07
4,3.0,0.011,6.0,950.0,9.0,14.5,5,7.45,ok,7.45,40.47,47.92,2.80,52.19,54.99


In [54]:
# trials_full

In [96]:
iter = 30
dictionary = trials_full[trials_full['iter']==iter].loc[iter-1,'gamma':'reg_lambda'].to_dict()
dictionary.update({'seed':42})
ntrees = dictionary['n_estimators']
learn_rate = dictionary['learning_rate']
dictionary['ntrees'] = int(ntrees)
dictionary['learn_rate'] = learn_rate
del dictionary['n_estimators']
del dictionary['learning_rate']
dictionary['max_depth'] = 2
dictionary

{'gamma': 12.0,
 'min_child_weight': 26.0,
 'reg_alpha': 1.0,
 'reg_lambda': 3.5,
 'seed': 42,
 'ntrees': 825,
 'learn_rate': 0.028,
 'max_depth': 2}

In [36]:
dictionary = {'gamma': 12.0,
 'min_child_weight': 26.0,
 'reg_alpha': 1.0,
 'reg_lambda': 3.5,
 'seed': 42,
 'ntrees': 825,
 'learn_rate': 0.028,
 'max_depth': 2}

In [37]:
best_model = H2OXGBoostEstimator(**dictionary)

In [38]:
best_model.train(y = bad_col, x = predictors, training_frame = Train_h2o, weights_column = wt_col)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1728625371034_35


Model Summary: 
    number_of_trees
--  -----------------
    825

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.020696591420484082
RMSE: 0.14386309957902368
LogLoss: 0.09910314691519784
Mean Per-Class Error: 0.3198687181327472
AUC: 0.7563099029590753
AUCPR: 0.056069018098655755
Gini: 0.5126198059181506

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.029864855110645294
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      2085  392  0.1583   (392.0/2477.0)
1      26    28   0.4815   (26.0/54.0)
Total  2111  420  0.1652   (418.0/2531.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value      idx
---------------------------  -----------  ---------  -----
max f1                       0.0298649    0.118143   0
max f2                       0.0298649    0.220126   0
max f0point5                 0.0298649    0.0807382  0
max accuracy                 0.0298649    0.834848   0
max precision                0.0298649    0.0666667  0
max recall                   0.0163625    1          8
max specificity              0.0298649    0.841744   0
max absolute_mcc             0.0287164    0.142932   1
max min_per_class_accuracy   0.0287164    0.685185   1
max mean_per_class_accuracy  0.0287164    0.717037   1
max tns                      0.0298649    2085       0
max fns                      0.0298649    26         0
max fps                      0.0163625    2477       8
max tps                      0.0163625    54         8
max tnr                      0.0298649    0.841744   0
max fnr                      0.0298649    0.481481   0
max fpr                      0.0163625    1          8
max tpr                      0.0163625    1          8

Gains/Lift Table: Avg response rate:  2.13 %, avg score:  2.17 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.165942                    0.0298649          3.12469   3.12469            0.0666667        0.0298649  0.0666667                   0.0298649           0.518519        0.518519                   212.469   212.469            0.360263
2        0.260371                    0.0287164          1.76499   2.63157            0.0376569        0.0287164  0.0561457                   0.0294483           0.166667        0.685185                   76.4993   163.157            0.434075
3        0.412485                    0.0227899          0.608706  1.88559            0.012987         0.022859   0.0402299                   0.0270184           0.0925926       0.777778                   -39.1294  88.559             0.373256
4        0.512841                    0.0214859          0.922645  1.69716            0.019685         0.0216739  0.0362096                   0.0259725           0.0925926       0.87037                    -7.73549  69.7155            0.365324
5        0.613196                    0.0206527          0.369058  1.4798             0.00787402       0.0206527  0.0315722                   0.0251019           0.037037        0.907407                   -63.0942  47.9799            0.300625
6        1                           0.0163625          0.239379  1                  0.00510725       0.0163862  0.0213354                   0.0217306           0.0925926       1                          -76.0621  0                  0

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse        trainin

### TRAIN

In [39]:
predict = best_model.predict(Train_h2o)
p =predict.as_data_frame()
dd = Train_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
# rnkOrdering2_ks_train_nw = rnkOrdering2_ks_code(pred, bad_col, "conwt","p1", 5)
print(ks_dict)
ks_df

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
{'Gini': 49.16, 'KS': 40.08}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.028716,0.029865,0.029668,507,475,32,6.31,20.03,19.18,59.26,40.08,3.343222
2,0.022790,0.028716,0.024623,506,498,8,1.58,40.02,39.28,74.07,34.79,0.882278
3,0.020653,0.022790,0.021296,506,497,9,1.78,60.02,59.35,90.74,31.39,0.349629
4,0.016362,0.020653,0.016688,506,503,3,0.59,80.01,79.65,96.30,16.65,1.078815
5,0.016362,0.016362,0.016362,506,504,2,0.40,100.00,100.00,100.00,0.00,1.236249


### TEST

In [40]:
predict = best_model.predict(Test_h2o)
p =predict.as_data_frame()
dd = Test_h2o[:,unique_cols + [wt_col] + [bad_col]].as_data_frame()
dd['RECWGT'] = dd[wt_col]
dd['conwt'] = 1
pred = pd.concat([dd.loc[:,(bad_col,"RECWGT","conwt")],p],axis=1)
ks_dict, ks_df = rnkOrderingN(pred, bad_col, "RECWGT","p1", 5)
print(ks_dict)
ks_df

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
{'Gini': 47.88, 'KS': 39.03}


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS,Bad_Rate_Diff
1,0.028716,0.029865,0.029501,218,207,11,5.05,20.07,19.47,47.83,28.36,2.099865
2,0.022790,0.028716,0.024303,217,210,7,3.23,40.06,39.23,78.26,39.03,0.799691
3,0.017026,0.022790,0.021077,217,213,4,1.84,60.04,59.27,95.65,36.38,0.267701
4,0.016362,0.017026,0.016396,217,217,0,0.00,80.02,79.68,95.65,15.97,1.639610
5,0.016362,0.016362,0.016362,217,216,1,0.46,100.00,100.00,100.00,0.00,1.176249


In [41]:
var_imp = best_model._model_json['output']['variable_importances'].as_data_frame()
var_imp['Var_unique'] = var_imp['variable'].str.split('.').str[0]
var_imp

,variable,relative_importance,scaled_importance,percentage,Var_unique
0,"pl_astcntev.[-inf, 1.0)",169.026093,1.000000,0.483424,pl_astcntev
1,"pi_inpaddrmailmatchflag.[1.0, inf)",132.388092,0.783241,0.378637,pi_inpaddrmailmatchflag
2,"pi_inpaddrnamepropmatchindx.[-99999, -99998, -99997]",24.214243,0.143257,0.069254,pi_inpaddrnamepropmatchindx
3,"pl_astemrgnewmsncev.[-99999, -99998, -99997]",24.015429,0.142081,0.068685,pl_astemrgnewmsncev


In [42]:
var_imp_list_baseline1 = list(var_imp['Var_unique'].unique())
print(len(var_imp_list_baseline1))
# print(var_imp_list_baseline1)

4


In [43]:
Var_Imp_DF = var_imp.groupby('Var_unique').agg({'percentage':'sum'}).reset_index().sort_values(by='percentage',ascending=False).reset_index(drop=True)
Var_Imp_DF

,Var_unique,percentage
0,pl_astcntev,0.483424
1,pi_inpaddrmailmatchflag,0.378637
2,pi_inpaddrnamepropmatchindx,0.069254
3,pl_astemrgnewmsncev,0.068685
